In [14]:
import pandas as pd
import os

print(os.getcwd())
path = os.getcwd() + '/openfoodfacts.csv'
print(path)
food_db = pd.read_csv(path, sep = '\t')

/home/sadu/Documents/Incubator-challenge
/home/sadu/Documents/Incubator-challenge/openfoodfacts.csv


/home/sadu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
usda_bfp = food_db.loc[food_db['creator'] == 'usda-ndb-import']
usda_bfp.shape

(169868, 163)

In [62]:
print(list(usda_bfp))
yogurts = usda_bfp[usda_bfp['product_name'].str.contains('Yogurt', na = False)]
print(yogurts.shape)

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 

In [101]:
from collections import defaultdict
import matplotlib.pyplot as plt
import re

ingredient_count = defaultdict(int)

for index, row in yogurts.iterrows():
    for ingredient in row['ingredients_text'].split(', '):
        ingredient_count[ingredient] += 1

df_count = pd.DataFrame.from_records([ingredient_count])


df_count['Cultured milk'] = 0

for column in df_count:
    if re.match(r'Cultured.*milk|milk.*Cultured', column):
        df_count['Cultured milk'] += df_count[column][0]
df_count = df_count.sort_values(by = [0], axis=1, ascending = False)
sorted(ingredient_count.items(), key=lambda x: x[1], reverse = True)

df_count

,Cultured milk,sugar,water,pectin,natural flavor,citric acid,natural flavors,l. acidophilus,l. bulgaricus,locust bean gum,...,fruit juice and annatto [color],fruit and vegetable live and active probiotic cultures: l. acidophilus,fruit juice [color] guar gum,"fruit juice (for color),",fruit juice & beta carotene (for color),fruit extract,fruit base (cane sugar,fruit and vegetables juice concentrate (for color),fruit and vegetable nutrient extract blend (orange extract,zucchini
0,1848,1579,1181,1154,995,750,672,644,538,528,...,1,1,1,1,1,1,1,1,1,1
